# SIADEX HTN ENGINE

imports requireds

In [1]:
!pip install ..
!pip install ../unified-planning

Defaulting to user installation because normal site-packages is not writeable
Processing /mnt/e/newDesktop/void/projects/siadex/up-siadex
  Preparing metadata (setup.py) ... done
  Created wheel for up-siadex: filename=up_siadex-0.0.1-py3-none-any.whl size=8668214 sha256=cb117e28e3ca58531c01b06b87941c569e3303f78ddfc7165a2a8a3dc153616d
  Stored in directory: /home/gorgue/.cache/pip/wheels/f5/68/44/88e2b79f736c596547038a3376e427061bae2bfda13b9591d5
Successfully built up-siadex
  Attempting uninstall: up-siadex
    Found existing installation: up-siadex 0.0.1
    Uninstalling up-siadex-0.0.1:
      Successfully uninstalled up-siadex-0.0.1
Defaulting to user installation because normal site-packages is not writeable
Processing /mnt/e/newDesktop/void/projects/siadex/up-siadex/unified-planning
  Preparing metadata (setup.py) ... done
  Created wheel for unified-planning: filename=unified_planning-0.4.2-py3-none-any.whl size=357569 sha256=87d8427c4d0b5cbe3d14b95242a466a14bae2271aa7f0f01d7c0a8

In [2]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus


## Registering the engine

In order to use `SIADEX`, we need to register it among the set of planning engines available for the UP library as follows.

In [3]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

## Reading a problem from a file (HDDL)

In [4]:
reader = PDDLReader()
problem = reader.parse_problem("../unified-planning/unified_planning/test/pddl/htn-transport/domain.hddl", "../unified-planning/unified_planning/test/pddl/htn-transport/problem.hddl")

## Debugging
Initialize SIADEX on debug mode.

In [5]:
siadex = OneshotPlanner(name='siadex')
debugger = siadex.debugger()
debugger.debug(problem)

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_env().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_1197/182887877.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE



The debugging process starts with the initial state of the problem and advance in the planning process with the nexp() or next() methods.

To get the actual state of the debugger we can use the method state(). This returns the list of fluents that are currently active.

In [6]:
state = debugger.state()
state

[road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

The agenda represents the actual process of the debugger expanding nodes. 
it returns a dict of nodes, each node has the following information:
 - subtask: the task/action that is susceptible to be expanded with the instanced object or named variables
 - status: TODO: Agenda...
 - expanded: If the node has been expanded or not
 - succesors: A list of the children nodes

In [7]:
nodes = debugger.agenda()
nodes

{0: 0: (root ) 
                 status: root, expanded: root, 
                 successors: [1, 2],
 1: 1: (deliver [package-0, city-loc-0]) 
                 status: agenda, expanded: unexpanded, 
                 successors: [],
 2: 2: (deliver [package-1, city-loc-2]) 
                 status: agenda, expanded: unexpanded, 
                 successors: []}

In [8]:
nodes[1].subtask

_t13: deliver(package-0, city-loc-0)
        time_constraints = [
        ]

With agenda_tree() we can take a look to the agenda in a easier way.

In [9]:
debugger.agenda_tree()

0: (root ) status: root, expanded: root
    1: (deliver [package-0, city-loc-0]) status: agenda, expanded: unexpanded
    2: (deliver [package-1, city-loc-2]) status: agenda, expanded: unexpanded


Advance one step in the debug process. 

In [10]:
debugger.nexp()

debug:> nexp
__________________________________________________
(*** 1 ***) Expanding: [0] (deliver package_0 city_loc_0)

(*** 1 ***) Selecting a candidate task.
(*** 1 ***) Found: 1 candidates (left).
      [0] :task (deliver ?p ?l)


With the method eval_precondotions() we can evaluate on the actual state the preconditions of an action.
It returns a list of valid groups of parameters. 

In [11]:
drive = problem.action("drive")
drive

action drive(vehicle - locatable v, location l1, location l2) {
    preconditions = [
      (at(v, l1) and road(l1, l2))
    ]
    effects = [
      at(v, l1) := false
      at(v, l2) := true
    ]
    simulated effect = None
  }

In [12]:
debugger.eval_preconditions(drive)

[{vehicle - locatable v: truck-0,
  location l1: city-loc-2,
  location l2: city-loc-1}]

In [13]:
debugger.state()

[road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

We get the fluent and objects from the original problem to work with them.

In [14]:
road = problem.fluent("road")
city_0 = problem.object("city-loc-0")
city_1 = problem.object("city-loc-1")
road, city_0, city_1

(bool road[l1=location, l2=location], city-loc-0, city-loc-1)

With add_break we can define breakpoints where the debugger will stop when exploring

In [15]:
debugger.add_break(road)

debug:> break (road ?l1 ?l2)
__________________________________________________


{0: {'id': 0, 'enabled': True, 'node': '(road ?l1 ?l2)'}}

In [16]:
debugger.add_break(drive)

debug:> break (drive ?v ?l1 ?l2)
__________________________________________________


{0: {'id': 0, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 1: {'id': 1, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

With enable_break and disable_break, we can enable a disable breakpoints

In [17]:
debugger.disable_break(0)

{0: {'id': 0, 'enabled': False, 'node': '(road ?l1 ?l2)'},
 1: {'id': 1, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

In [18]:
debugger.enable_break(0)

{0: {'id': 0, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 1: {'id': 1, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

In [19]:
debugger.list_break()

{0: {'id': 0, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 1: {'id': 1, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

With an expression we can alter the live state of the debugger 

In [20]:
f_road = road(city_0, city_1)
f_road.Not()

(not road(city-loc-0, city-loc-1))

In [21]:
# Actual state
debugger.state()
# We can see road(city-loc-0, city-loc-1) on the state

[road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

With apply_effect we can modify the state

In [22]:
debugger.apply_effect(f_road.Not())

[road(city-loc-1, city-loc-0),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

In [23]:
debugger.apply_effect(f_road)

[road(city-loc-1, city-loc-0),
 road(city-loc-0, city-loc-1),
 road(city-loc-1, city-loc-2),
 road(city-loc-2, city-loc-1),
 at(package-0, city-loc-1),
 at(package-1, city-loc-1),
 at(truck-0, city-loc-2),
 capacity(truck-0, capacity-1),
 capacity-predecessor(capacity-0, capacity-1)]

With eval_preconditions we can obtain a list of valid parameters of a fluent or action

In [24]:
debugger.eval_preconditions(road)

[{'l1': city-loc-2, 'l2': city-loc-1},
 {'l1': city-loc-1, 'l2': city-loc-2},
 {'l1': city-loc-0, 'l2': city-loc-1},
 {'l1': city-loc-1, 'l2': city-loc-0}]

In [25]:
debugger.eval_preconditions(drive)

[{vehicle - locatable v: truck-0,
  location l1: city-loc-2,
  location l2: city-loc-1}]

Avance 8 tasks in the planning process

In [26]:
debugger.nexp(10)

debug:> nexp
__________________________________________________
(*** 1 ***) Selecting a method to expand from compound task.
      :task (deliver ?p ?l)
(*** 1 ***) Found: 1 methods to expand (left).
      [0] 
      (:method m_deliver
      :precondition
      ( )
      :tasks (
         (get_to ?v ?l1)
         (load ?v ?l1 ?p)
         (get_to ?v ?l2)
         (unload ?v ?l2 ?p)
      )
      )
Breakpoint 0 reached.
debug:> nexp
__________________________________________________
(*** 1 ***) Expanding method: m_deliver
(*** 1 ***) Working in task:
(:task deliver
   :parameters ( ?p - package ?l - location)
   (:method m_deliver
   :precondition
   ( )
   :tasks (
      (get_to ?v ?l1)
      (load ?v ?l1 package_0)
      (get_to ?v ?l2)
      (unload ?v ?l2 package_0)
   )
   )
)

(*** 1 ***) Using method: m_deliver
(*** 1 ***) No preconditions.
Selecting unification: 
(*** 2 ***) Depth: 2
(*** 2 ***) Selecting task to expand from agenda.
Breakpoint 0 reached.
debug:> nexp
___________

With plan we can obtain the result plan that is already discovered  

In [27]:
debugger.plan()

[drive(truck-0, city-loc-2, city-loc-1)]

In [28]:
debugger.agenda_tree()

0: (root ) status: root, expanded: root
    2: (deliver [package-1, city-loc-2]) status: agenda, expanded: unexpanded
    8: (drive [truck-0, city-loc-2, city-loc-1]) status: pending, expanded: action
        4: (load [truck-0, l, package-0]) status: pending, expanded: unexpanded
            5: (get-to [truck-0, l]) status: pending, expanded: unexpanded
                6: (unload [truck-0, l, package-0]) status: pending, expanded: unexpanded


In [29]:
debugger.continue_to(drive)

debug:> break (drive ?v ?l1 ?l2)
__________________________________________________
debug:> continue
__________________________________________________
(*** 4 ***) Expanding method: m_load
(*** 4 ***) Working in task:
(:task load
   :parameters ( ?v - vehicle ?l - location ?p - package)
   (:method m_load
   :precondition
   ( )
   :tasks (
      (pick_up truck_0 ?l1 package_0 ?s1 ?s2)
   )
   )
)

(*** 4 ***) Using method: m_load
(*** 4 ***) No preconditions.
Selecting unification: 
(*** 5 ***) Depth: 5
(*** 5 ***) Selecting task to expand from agenda.
Breakpoint 0 reached.


With force_run we can directly interact with the debugger using string commands

In [30]:
debugger.force_run("help break")

debug:> help break
__________________________________________________
Command: `break'. Shortcut `b'

Description: Manages the stablished breakpoints.
	`break':	Lists all defined breakpoints.
	`break <number>':	Prints breakpoint whith given id.
	`break <predicate>':	Defines a new breakpoint. <predcate> can be a task definition or a simple predicate.
See also: `watch', `disable', `enable'.


In [31]:
debugger.list_break()

{0: {'id': 0, 'enabled': True, 'node': '(road ?l1 ?l2)'},
 1: {'id': 1, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'},
 2: {'id': 2, 'enabled': True, 'node': '(drive ?v ?l1 ?l2)'}}

Lets disable all the breakpoints

In [32]:
for i,_ in debugger.list_break().items():
    debugger.disable_break(i)

In [33]:
debugger.continue_run()

debug:> continue
:action (drive truck_0 city_loc_2 city_loc_1)
:action (pick_up truck_0 city_loc_1 package_0 capacity_0 capacity_1)
:action (drive truck_0 city_loc_1 city_loc_0)
:action (drop truck_0 city_loc_0 package_0 capacity_0 capacity_1)
:action (drive truck_0 city_loc_0 city_loc_1)
:action (pick_up truck_0 city_loc_1 package_1 capacity_0 capacity_1)
:action (drive truck_0 city_loc_1 city_loc_0)
:action (drop truck_0 city_loc_0 package_1 capacity_0 capacity_1)
__________________________________________________
(*** 5 ***) Expanding: [8] (pick_up truck_0 ?l1 package_0 ?s1 ?s2)

(*** 5 ***) Selecting a candidate task.
(*** 5 ***) Found: 1 candidates (left).
      [0] :action (pick_up truck_0 ?l1 package_0 ?s1 ?s2)
(*** 5 ***) Solving action:
(:action pick_up
 :parameters ( truck_0 ?l1 - location package_0 ?s1 - capacity_number ?s2 - capacity_number)
 :precondition
   (and
      (and
         (at_ truck_0 ?l1)
         (at_ package_0 ?l1)
         (capacity_predecessor ?s1 ?s2)
    

In [35]:
debugger.plan()

Exception: Debugger stopped

In [36]:
debugger.help()

Exception: Debugger stopped

In [37]:
debugger.stop()

Debugger stopped


### END DEBUGGER